# Modelado de tópicos

El modelado de tópicos es una tarea de NLP que busca, dado un *corpus* de documentos,

1. Descubrir los distintos *tópicos* latentes en el *corpus*, y
2. Entender cada documento como una combinación de distintos tópicos.

Antes de explicar qué es un tópico, veamos el modelo de datos. Tenemos lo siguiente:

1. Un cuerpo $D$ de $N$ documentos $d_1, \ldots, d_N$
2. Cada documento $d_i$ es una colección de palabras, donde $d_{i,j} \ge 0$ es el número de veces que la palabra $w_j$ aparece en el documento $d_i$ (representación **bag of words**)
3. Al conjunto total de $W$ palabras $w_1, w_2, \ldots, w_W$ de todo el cuerpo se le conoce como **diccionario**.

### Ejemplo mínimo

**Corpus**
> $d_1$ = ``Como poco coco como, poco coco compro. Como compro poco coco, poco coco como``<br>
> $d_2$ = ``A Cuesta le cuesta subir la cuesta, y en medio de la cuesta va y se acuesta``

**Diccionario**
> ``como, poco, coco, compro, a, cuesta, le, subir, la, y, en, medio, de, va, se, acuesta`` <br>
> $W = 16$

**Representación bag-of-words**
> $d_1$ = ``{"coco": 4, "como": 4, "compro": 2, "poco": 4}`` <br>
> $d_2$ = ``{"a": 1, "acuesta": 1, "cuesta": 4, "de": 1, "en": 1, "la": 2, "le": 1, "medio": 1, "se": 1, "subir": 1, "va": 1, "y": 2}``

Entonces, un tópico $t$ es una combinación lineal de palabras del diccionario, de modo que $\sum_{j=1}^{m} t_j = 1$.


En términos matriciales, la entrada de nuestro método será una matriz $\mathbf{D}$ de dimension $N \times W$. Esta matriz muy seguramente será **dispersa**. Es decir, que la mayoría de sus valores serán 0. 

||como|poco|coco|compro|a|cuesta|le|subir|la|y|en|medio|de|va|se|acuesta|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$d_1$|4|4|4|2|0|0|0|0|0|0|0|0|0|0|0|0|
|$d_2$|0|0|0|0|1|4|1|1|2|1|1|1|1|1|1|1|

**Importante**: la información secuencial (cuál palabra sigue a cuál otra) se pierde en la representación *bag-of-words*. Luego estudiaremos otros modelos para esto. 

Finalmente, nuestro método de modelado de tópicos toma un valor $K$ (el número de tópicos a modelar) y va a producir lo siguiente:

1. Una matriz de tópicos-palabras $T$ de dimensiones $K \times W$
 - Es decir, de qué palabras se compone cada tópico (fila) $T_i$
 - De modo que $\sum_{i=1}^W T_i = 1$

 
2. Una matriz de documentos-tópicos $P$ de dimensiones $N \times K$
 - Es decir, la fila $P_i$ nos explica de qué tópicos se compone el documento $d_i$
 - Igualmente, $\sum_{i=1}^K P_i = 1$

## Asignación Latente de Dirichlet (LDA)

Uno de los mejores modelos de tópicos es la Asignación Latente de Dirichlet (LDA). 

Este modelo presenta una mejoría sobre métodos anteriores (como pLSA) en que es un modelo **bayesiano** que incorpora la siguiente suposición como información prior: **dentro de un *corpus*, la correspondencia entre documentos, palabras y tópicos será *dispersa***. 

Es decir, que un documento usualmente corresponde a **pocos tópicos**, y que los tópicos están compuestos de **pocas palabras**. Entonces, la gran mayoría de los elementos en las matrices $T$ y $P$ serán 0. 

### Dataset: 20 newsgroups

Nuestro dataset de ejemplo será el conocido **20 newsgroups**. Este dataset contiene mensajes de 20 grupos de discusión de Usenet, divididos en varios tópicos, incluyendo política, religión, automovilismo, béisbol, computación, etc. 


In [1]:
# Obtenemos el dataset 
import sklearn.datasets
tng = sklearn.datasets.fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Vemos el listado de grupos de discusión
print('*** Grupos de discusión: \n')
print(*tng['target_names'], sep='\n')

# Vemos un mensaje de ejemplo
print('\n\n*** Mensajes de ejemplo: \n\n', tng['data'][0], '\n\n---\n\n', tng['data'][50])

*** Grupos de discusión: 

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


*** Mensajes de ejemplo: 

 

I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was 


### Preparación de datos

Uno de los aspectos más importantes a la hora de trabajar con un dataset para análisis de lenguaje natural es el preprocesamiento, el cual incluye la eliminación de información irrelevante como signos de puntuación, diferencias entre mayúsculas y minúsculas y **"stop words"**, es decir, palabras como conjunciones ("to", "a", "it"). La clase que utilizaremos para convertir los documentos en su representación *bag-of-words*, llamada ``sklearn.feature_extraction.text.CountVectorizer``, trae consigo la facilidad para eliminar stop words y símbolos de puntuación. Veamos

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

## Creamos una instancia de CountVectorizer usando stop words en inglés
## y la "entrenaremos" con el primer documento para ver cómo resultan
cv = CountVectorizer(stop_words='english', token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z][a-zA-Z]+\b").fit([tng['data'][0]])

print('*** ANTES: ', tng['data'][0])
print('*** DESPUES:\n\n',' '.join(cv.build_analyzer()(tng['data'][0])))

*** ANTES:  

I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


*** DESPUES:

 sure bashers pens fans pretty confused lack kind posts recent pens massacre devils actually bit puzzled bit relieved going end non pittsburghers relief bit praise pens man killing devils worse thought jagr just showed better regular season stats lot fun watch 

Ahora, procederemos a realizar esto mismo con el dataset entero.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

## Tokenizamos el dataset entero (toma unos diez segundos en ejecutarse)
cv = CountVectorizer(stop_words='english', token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z][a-zA-Z]+\b").fit(tng['data'])
tng_matrix = cv.transform(tng['data'])

### Ejecutando LDA con ``scikit-learn``

La clase de interés es ``sklearn.decomposition.LatentDirichletAllocation``. 

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

## Entrenamos el modelo LDA (toma un par de minutos)
lda = LatentDirichletAllocation(n_components=20, verbose=True).fit(tng_matrix)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [5]:
# Y, ahora, vamos a ver cómo se ven los tópicos. 
import numpy as np
comp = lda.components_
vec = np.array(cv.get_feature_names())
for i in range(0, comp.shape[0]):
    print('*** TOPICO ', i)
    print(vec[comp[i].argsort()[-20:][::-1]])
    print('\n')


*** TOPICO  0
['government' 'people' 'war' 'rights' 'state' 'states' 'turkish' 'right'
 'public' 'united' 'national' 'law' 'security' 'american' 'privacy'
 'armenian' 'information' 'new' 'world' 'muslims']


*** TOPICO  1
['gun' 'fbi' 'law' 'police' 'guns' 'people' 'koresh' 'don' 'crime'
 'weapons' 'batf' 'government' 'think' 'firearms' 'control' 'use' 'like'
 'just' 'did' 'illegal']


*** TOPICO  2
['game' 'team' 'games' 'year' 'play' 'season' 'hockey' 'players' 'win'
 'league' 'player' 'good' 'baseball' 'don' 'period' 'pit' 'teams' 'think'
 'hit' 'nhl']


*** TOPICO  3
['know' 'thanks' 'msg' 'food' 'mail' 'don' 'time' 'list' 'number' 'san'
 'does' 'just' 'address' 'group' 'battery' 'looking' 'chinese' 'points'
 'let' 'com']


*** TOPICO  4
['won' 'lost' 'mov' 'slave' 'use' 'period' 'health' 'drive' 'seattle'
 'san' 'blue' 'number' 'master' 'new' 'rate' 'april' 'power' 'vancouver'
 'red' 'national']


*** TOPICO  5
['people' 'don' 'just' 'think' 'like' 'say' 'know' 'does' 'way' 'point

In [6]:
## Ahora, veamos la composición de tópicos de algún documento

print(tng['data'][1234])
tx = lda.transform(tng_matrix[1234])

print('TOPICOS (en orden):')
print(tx.argsort()[0][::-1])

For Sale:

Fujitsu 324meg SCSI drive.  $450

Maxtor 338meg ESDI drive.  $425

Maxtor 160meg ESDI drive.  $225

Toshiba 106meg IDE drive.  $175

XT case & motherboard.  $50

DTC 16-bit MFM 2HD 2FD controler.  $30

All items are used, in full working condition, and have a  
warranty for one week unless otherwise specified.  All prices 
are %100 negotiable, shipping not included. 

Wanted:  

Developers kit for SB
17" SVGA moniters (two of them).


TOPICOS (en orden):
[19  8  4  9  7 10 18 15  1 17  5  6 16  0  3  2 11 12 14 13]
